In [8]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv()

True

In [9]:
llm = ChatGroq(model="openai/gpt-oss-20b")

In [28]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2
import json

In [11]:
RESPONSE_JSON = {
    "1":{
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
    "2":{
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
    "3":{
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
} 

In [12]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to create a quiz of {number} 
multiple choice questions for {subject} students in {tone} tone. Make sure the questions are 
not repeated and check all the questions to be conforming the text as well. Make sure to format 
your response like RESPONSE_JSON below and use it as a guide. Ensure to make {number} MCQs
### RESPONSE_JSON {response_json}
"""

In [16]:
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.
You need to evaluate the complexity of the question and give a complete analysis of the quiz. 
Only use at max 50 words for complexity. If the quiz is not as per with the congnitive and analytical 
abilities of the students, updata the quiz questions which needs to be changed the tone such that it 
perfectly fits the student abilities.
Quiz_MCQs: {quiz} 
"""

In [46]:
# tone means complexity of quiz like, easy, moderate, hard
prompt = PromptTemplate(
    template=TEMPLATE,
    input_variables=["text", "number", "tone", "response_json", "subject"]
)

In [47]:
prompt2 = PromptTemplate(
    template=TEMPLATE2,
    input_variables=["quiz"]
)

In [15]:
quiz_chain = LLMChain(llm=llm, prompt=prompt, output_key="quiz", verbose=True)

In [18]:
review_chain = LLMChain(llm=llm, prompt=prompt2, output_key="review", verbose=True)

In [20]:
genreate_evaluate_chain = SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"], output_variables=["quiz", "review"], verbose=True)

In [25]:
file_path = "D:/python/DL/GenAi/Project/MCQGEN/data.txt"

In [26]:
with open(file=file_path, mode='r') as f:
    TEXT = f.read()

In [27]:
print(TEXT)

The Forgotten Kingdom of Aranthor
Long ago, when the stars seemed closer to the earth and the rivers spoke in whispers to the wind, there stood a kingdom known as Aranthor. Hidden between towering mountains of silver rock and boundless forests that stretched beyond the horizon, it was a land of prosperity and legend. For centuries, its name echoed in the songs of travelers, its banners painted in golden lions upon crimson silk fluttered in the breeze, and its people thrived in harmony with the land. But like all great kingdoms, the shadow of time would one day swallow it, leaving behind only fragments of memory.

The story begins with King Elandor, the last great ruler of Aranthor. He was not merely a sovereign who sat upon a throne of carved obsidian; he was a warrior of unyielding courage and a philosopher who sought wisdom in the scrolls of the ancients. His people adored him, for he walked among them often, dressed not in jewel-studded robes but in simple linen, listening to the co

In [29]:
NUMBER=5
SUBJECT="Kingdom"
TONE="simple"

In [ ]:
# Below function is use to track no.of tokens and cost according to input and output but, this function is use only for openai models(llms)

# with get_openai_callback() as cb:
#     response = genreate_evaluate_chain(
#         {
#             "text": TEXT,
#             "number": NUMBER,
#             "subject": SUBJECT,
#             "tone":TONE,
#             "response_json": json.dump(RESPONSE_JSON)
#         }
#     )

# print(f"Total Tokens: {cb.total_tokens}") # total number of tokens
# print(f"Prompt tokens: {cb.prompt_tokens}") # input tokens
# print(f"Completion Tokens: {cb.completion_tokens}") # output tokens
# print(f"Total Cost: {cb.total_cost}") # total cost

In [32]:
response1 = genreate_evaluate_chain.invoke({
    "text": TEXT,
    "number": NUMBER,
    "subject": SUBJECT,
    "tone":TONE,
    "response_json": json.dumps(RESPONSE_JSON)
})
response1



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The Forgotten Kingdom of Aranthor
Long ago, when the stars seemed closer to the earth and the rivers spoke in whispers to the wind, there stood a kingdom known as Aranthor. Hidden between towering mountains of silver rock and boundless forests that stretched beyond the horizon, it was a land of prosperity and legend. For centuries, its name echoed in the songs of travelers, its banners painted in golden lions upon crimson silk fluttered in the breeze, and its people thrived in harmony with the land. But like all great kingdoms, the shadow of time would one day swallow it, leaving behind only fragments of memory.

The story begins with King Elandor, the last great ruler of Aranthor. He was not merely a sovereign who sat upon a throne of carved obsidian; he was a warrior of unyielding courage and a philosopher who sought wisdom in the scrolls of the ancients. His people adored him,

{'text': 'The Forgotten Kingdom of Aranthor\nLong ago, when the stars seemed closer to the earth and the rivers spoke in whispers to the wind, there stood a kingdom known as Aranthor. Hidden between towering mountains of silver rock and boundless forests that stretched beyond the horizon, it was a land of prosperity and legend. For centuries, its name echoed in the songs of travelers, its banners painted in golden lions upon crimson silk fluttered in the breeze, and its people thrived in harmony with the land. But like all great kingdoms, the shadow of time would one day swallow it, leaving behind only fragments of memory.\n\nThe story begins with King Elandor, the last great ruler of Aranthor. He was not merely a sovereign who sat upon a throne of carved obsidian; he was a warrior of unyielding courage and a philosopher who sought wisdom in the scrolls of the ancients. His people adored him, for he walked among them often, dressed not in jewel-studded robes but in simple linen, listen

In [39]:
print(response1.get("quiz"))

```json
{
  "1": {
    "mcq": "Who was the last great ruler of Aranthor?",
    "options": {
      "a": "Malgareth",
      "b": "Sevrik",
      "c": "Elandor",
      "d": "Elyndra"
    },
    "correct": "c"
  },
  "2": {
    "mcq": "What did King Elandor wear when he walked among his people?",
    "options": {
      "a": "Jewel‑studded robes",
      "b": "Simple linen",
      "c": "Golden lion cloak",
      "d": "Obsidian armor"
    },
    "correct": "b"
  },
  "3": {
    "mcq": "Which kingdom allied with the Ebon Serpents against Aranthor?",
    "options": {
      "a": "Drevarn",
      "b": "Veylara",
      "c": "Aranthor",
      "d": "Seliane"
    },
    "correct": "a"
  },
  "4": {
    "mcq": "What artifact did King Elandor seek to defeat the enemies?",
    "options": {
      "a": "Crown of Solaris",
      "b": "Sword of Dawn",
      "c": "Shield of Veylara",
      "d": "Tome of Elders"
    },
    "correct": "a"
  },
  "5": {
    "mcq": "What happened to King Elandor after wearing th

In [41]:
quiz = response1.get("quiz")
clean_quiz = quiz.replace("```json", "").replace("```", "").strip()

quiz_json = json.loads(clean_quiz)
quiz_json

{'1': {'mcq': 'Who was the last great ruler of Aranthor?',
  'options': {'a': 'Malgareth', 'b': 'Sevrik', 'c': 'Elandor', 'd': 'Elyndra'},
  'correct': 'c'},
 '2': {'mcq': 'What did King Elandor wear when he walked among his people?',
  'options': {'a': 'Jewel‑studded robes',
   'b': 'Simple linen',
   'c': 'Golden lion cloak',
   'd': 'Obsidian armor'},
  'correct': 'b'},
 '3': {'mcq': 'Which kingdom allied with the Ebon Serpents against Aranthor?',
  'options': {'a': 'Drevarn', 'b': 'Veylara', 'c': 'Aranthor', 'd': 'Seliane'},
  'correct': 'a'},
 '4': {'mcq': 'What artifact did King Elandor seek to defeat the enemies?',
  'options': {'a': 'Crown of Solaris',
   'b': 'Sword of Dawn',
   'c': 'Shield of Veylara',
   'd': 'Tome of Elders'},
  'correct': 'a'},
 '5': {'mcq': 'What happened to King Elandor after wearing the Crown of Solaris?',
  'options': {'a': 'He became immortal',
   'b': 'He lost his sight',
   'c': 'He drained his life force and died',
   'd': 'He turned into a dragon

## Or


In [2]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
llm = ChatGroq(model="openai/gpt-oss-20b")

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain.schema.runnable import RunnableLambda
from langchain_core.output_parsers import JsonOutputParser 
import PyPDF2
import json

In [5]:
RESPONSE_JSON = {
    "1":{
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
    "2":{
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
    "3":{
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
} 

In [6]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to create a quiz of {number} 
multiple choice questions for {subject} students in {tone} tone. Make sure the questions are 
not repeated and check all the questions to be conforming the text as well. Make sure to format 
your response like RESPONSE_JSON below and use it as a guide. Ensure to make {number} MCQs
### RESPONSE_JSON {response_json}
"""

In [7]:
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.
You need to evaluate the complexity of the question and give a complete analysis of the quiz. 
Only use at max 50 words for complexity. If the quiz is not as per with the congnitive and analytical 
abilities of the students, updata the quiz questions which needs to be changed the tone such that it 
perfectly fits the student abilities.
Quiz_MCQs: {quiz} 
"""

In [8]:
# tone means complexity of quiz like, easy, moderate, hard
prompt1 = PromptTemplate(
    template=TEMPLATE,
    input_variables=["text", "number", "tone", "response_json", "subject"]
)

In [9]:
prompt2 = PromptTemplate(
    template=TEMPLATE2,
    input_variables=["quiz", "subject"]
)

In [10]:
def extract_content(ai_message, subject=None):
    return {"quiz": ai_message.content, "subject": subject}

prompt3 = RunnableLambda(extract_content)

In [11]:
parser = JsonOutputParser()

In [12]:
# or
quiz_chain = prompt1 | llm
review_chain = prompt2 | llm
final_chain = quiz_chain | prompt3 | review_chain | parser

In [13]:
file_path = "D:/python/DL/GenAi/Project/MCQGEN/data.txt"

In [14]:
with open(file=file_path, mode='r') as f:
    TEXT = f.read()

In [15]:
NUMBER=5
SUBJECT="Kingdom"
TONE="simple"

In [20]:
response2 = final_chain.invoke({
    "text": TEXT,
    "number": NUMBER,
    "subject": SUBJECT,
    "tone":TONE,
    "response_json": json.dumps(RESPONSE_JSON)
})
response2

{'1': {'mcq': 'Who was the last king of Aranthor?',
  'options': {'a': 'Malgareth', 'b': 'Elandor', 'c': 'Sevrik', 'd': 'Elyndra'},
  'correct': 'b'},
 '2': {'mcq': 'What was the name of the secret group that tried to overthrow Aranthor?',
  'options': {'a': 'Ebon Serpents',
   'b': 'Silver Lions',
   'c': 'Golden Lions',
   'd': 'Night Watch'},
  'correct': 'a'},
 '3': {'mcq': 'Which item did King Elandor want to win the war?',
  'options': {'a': 'Crown of Solaris',
   'b': 'Sword of Dawn',
   'c': 'Shield of Veylara',
   'd': 'Orb of Light'},
  'correct': 'a'},
 '4': {'mcq': 'Who commanded the cavalry with the golden lion flag?',
  'options': {'a': 'Lady Seliane',
   'b': 'King Elandor',
   'c': 'Malgareth',
   'd': 'Sevrik'},
  'correct': 'a'},
 '5': {'mcq': 'What happened to Aranthor after the last battle?',
  'options': {'a': 'It grew stronger',
   'b': 'It was taken over by neighbors',
   'c': 'It became a floating city',
   'd': 'It disappeared into the sky'},
  'correct': 'b'}}

In [21]:
quiz_table_data = []

for i in response2:
    question = response2[i]['mcq']
    options = response2[i]["options"]
    option_str = ""
    for j, k in options.items():
        option_str = option_str + f" {j}: {k} |"
    corect_answer = response2[i]["correct"]
    
    quiz_table_data.append({"MCQ": question, "Choices": option_str[:-1], "Correct": corect_answer})

In [22]:
quiz_table_data

[{'MCQ': 'Who was the last king of Aranthor?',
  'Choices': ' a: Malgareth | b: Elandor | c: Sevrik | d: Elyndra ',
  'Correct': 'b'},
 {'MCQ': 'What was the name of the secret group that tried to overthrow Aranthor?',
  'Choices': ' a: Ebon Serpents | b: Silver Lions | c: Golden Lions | d: Night Watch ',
  'Correct': 'a'},
 {'MCQ': 'Which item did King Elandor want to win the war?',
  'Choices': ' a: Crown of Solaris | b: Sword of Dawn | c: Shield of Veylara | d: Orb of Light ',
  'Correct': 'a'},
 {'MCQ': 'Who commanded the cavalry with the golden lion flag?',
  'Choices': ' a: Lady Seliane | b: King Elandor | c: Malgareth | d: Sevrik ',
  'Correct': 'a'},
 {'MCQ': 'What happened to Aranthor after the last battle?',
  'Choices': ' a: It grew stronger | b: It was taken over by neighbors | c: It became a floating city | d: It disappeared into the sky ',
  'Correct': 'b'}]

In [23]:
import pandas as pd
quiz = pd.DataFrame(quiz_table_data)
quiz

,MCQ,Choices,Correct
0,Who was the last king of Aranthor?,a: Malgareth | b: Elandor | c: Sevrik | d: El...,b
1,What was the name of the secret group that tri...,a: Ebon Serpents | b: Silver Lions | c: Golde...,a
2,Which item did King Elandor want to win the war?,a: Crown of Solaris | b: Sword of Dawn | c: S...,a
3,Who commanded the cavalry with the golden lion...,a: Lady Seliane | b: King Elandor | c: Malgar...,a
4,What happened to Aranthor after the last battle?,a: It grew stronger | b: It was taken over by...,b


In [24]:
quiz.to_csv("kingdom.csv", index=False)